on local pc set file structure like this

```
DLSprint_challenge
│   README.md
│   .gitignore    
│
└───data
│   │   dlsprint.zip
│   └───dlsprint
│       │   ...
│   
└───scripst
│       │   ...
└───starter_scripts 
│       │   ... 
```

In [99]:
# from google.colab import drive
# drive.mount('/content/gdrive')
# drive.mount("/content/gdrive", force_remount=True)

In [100]:
# !pip install kaggle
# /mnt/e/kaggle/DLSprint_challenge/.gitignore

In [101]:
# root_path = '/content/gdrive/MyDrive/kaggleProject'  #change dir to your project folder

In [102]:
# from google.colab import files
# files.upload()

In [103]:
!pwd

/mnt/c/Users/tonmo/Desktop/DLSprint_challenge/scripst


In [104]:
# !mkdir -p ~/.kaggle
# # !cp kaggle.json ~/.kaggle/
# !cp /content/gdrive/MyDrive/kaggleProject/kaggle.json ~/.kaggle/ # for use on tonmoy's gdrive
# !ls ~/.kaggle
# !chmod 600 /root/.kaggle/kaggle.json 

In [105]:
# !kaggle competitions download -c dlsprint
# !kaggle competitions download -c dlsprint -p /content/gdrive/MyDrive/kaggleProject/dlsprint

In [106]:
# import zipfile
# # extract_path = '/content/input/dlsprint'
# extract_path = 'data/dlsprint'
# with zipfile.ZipFile('data/dlsprint.zip', 'r') as zip_ref:
#     zip_ref.extractall(extract_path)

In [107]:
import unicodedata
text="বায়ান্ন"
new_text = unicodedata.normalize("NFKC", text)
print("text:",text)
print("new text:",new_text)
print("are they same?:",text==new_text)

text: বায়ান্ন
new text: বায়ান্ন
are they same?: False


In [108]:
print("first one:",[f for f in 'কেন্দ্রীয়'])
print("second one:",[f for f in 'কেন্দ্রীয়'])

first one: ['ক', 'ে', 'ন', '্', 'দ', '্', 'র', 'ী', 'য', '়']
second one: ['ক', 'ে', 'ন', '্', 'দ', '্', 'র', 'ী', 'য়']


In [109]:
print("first one:",[f for f in text])
print("second one:",[f for f in new_text])

first one: ['ব', 'া', 'য়', 'া', 'ন', '্', 'ন']
second one: ['ব', 'া', 'য', '়', 'া', 'ন', '্', 'ন']


## Importhing Necessary Libraries for EDA

In [110]:
# !conda install -c conda-forge librosa -y
# conda install -c conda-forge ipywidgets -y
# conda install -c conda-forge pandas-profiling -y

In [111]:
import os

import numpy as np
import pandas as pd

import librosa
import librosa.display

import IPython.display as ipd # for displaying the play audio buttom

import warnings
warnings.filterwarnings('ignore')

## Reading the .csv files

In [112]:
extract_path = '../data/dlsprint'
os.path.join(extract_path,)
train_df = pd.read_csv(os.path.join(extract_path,'train.csv'))
val_df = pd.read_csv(os.path.join(extract_path,'validation.csv'))
sample_submissin_df = pd.read_csv(os.path.join(extract_path,'sample_submission.csv'))

## Take a look at the .csv files

In [113]:
train_df.head(n = 3)

,client_id,path,sentence,up_votes,down_votes,age,gender,accents,locale
0,6c274d3678353b62f5d3465c771e4a8c5cd917186a5323...,common_voice_bn_30991326.mp3,বাবা সত্যেন ঘোষ।,1.0,1.0,NaN,NaN,NaN,bn
1,3991c2cfe1be64e77f2f6bdaeaae601219af906c1295cd...,common_voice_bn_30991371.mp3,দেওয়ান ফরিদ গাজী বাংলাদেশ আওয়ামী লীগের উপদেষ...,1.0,0.0,NaN,NaN,NaN,bn
2,3991c2cfe1be64e77f2f6bdaeaae601219af906c1295cd...,common_voice_bn_30991410.mp3,এর মাধ্যমে কারও গান গাওয়া দক্ষতা বা নিজস্ব ধর...,0.0,0.0,NaN,NaN,NaN,bn


In [114]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206950 entries, 0 to 206949
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   client_id   206950 non-null  object 
 1   path        206950 non-null  object 
 2   sentence    206950 non-null  object 
 3   up_votes    206950 non-null  float64
 4   down_votes  206950 non-null  float64
 5   age         60309 non-null   object 
 6   gender      61196 non-null   object 
 7   accents     11088 non-null   object 
 8   locale      206950 non-null  object 
dtypes: float64(2), object(7)
memory usage: 14.2+ MB


In [115]:
train_df.isna().sum()

client_id          0
path               0
sentence           0
up_votes           0
down_votes         0
age           146641
gender        145754
accents       195862
locale             0
dtype: int64

In [116]:
train_df['down_votes'].unique()

array([ 1.,  0.,  5.,  2.,  3.,  6.,  4.,  7., 14., 10.,  9., 13., 16.,
        8., 21., 17., 12., 11., 22., 24., 30., 18., 48., 20.])

In [117]:
train_df['up_votes'].unique()

array([  1.,   0.,   6.,   2.,   4.,   3.,  16.,   7.,   8.,   5.,  14.,
        11.,  15.,  10.,  12.,  13.,  17.,   9.,  18.,  20.,  30.,  26.,
        24.,  22.,  32.,  28.,  29.,  21.,  53.,  34.,  23.,  38.,  35.,
        19.,  36.,  27.,  40.,  25.,  37.,  31.,  45.,  39.,  50.,  33.,
        47.,  42.,  46.,  66.,  59.,  78.,  56.,  44., 108.,  88.,  55.,
        43.,  41.,  60.,  92., 124.,  51.,  54.,  52.,  48.,  68.,  62.,
       106.,  58.,  80.,  74.,  70.,  84.,  94.])

Higher the upvotes more accurate the text labels

Samples with higher number of downvote than upper are considered as incorrect.

## Let't check how label them

In [118]:
train_df['locale'].unique()

array(['bn'], dtype=object)

So all the audio files are labled by native bengalies

*Columns named 'age', 'gender', 'accents', 'locale' would be removed as they didn't carry important information for the project.*

In [119]:
train_df = train_df.drop(['age', 'gender', 'accents', 'locale'],axis = 1)
val_df = val_df.drop(['age', 'gender', 'accents', 'locale'],axis = 1)

In [120]:
# librosa.load() takes the path and returns a numpy array and the sample rate of the audio.
# extract_path = '../data/dlsprint'
# data/dlsprint.zip
AUDIO_PATH= os.path.join(extract_path,'train_files','common_voice_bn_30614352.mp3')
print(AUDIO_PATH)
# AUDIO_PATH = 'data/dlsprint/train_files/common_voice_bn_30614352.mp3'
audio, sr = librosa.load(AUDIO_PATH)

print('Shape of the audio: ', audio.shape)
print('Sample rate of the audio: ', sr)
ipd.display(ipd.Audio(data=audio, rate=sr))
print('Audio Label :',train_df[train_df['path'] == os.path.split(AUDIO_PATH)[1]].sentence)

../data/dlsprint/train_files/common_voice_bn_30614352.mp3
Shape of the audio:  (107163,)
Sample rate of the audio:  22050


Audio Label : 195102    বাকি অর্ধেক ডানা সাদা বর্ণের।
Name: sentence, dtype: object


## All the audio files are in .mp3 format we need to convert them into .wav format for faster processing.

In [121]:
# !pip install pydub

In [122]:
import os
import skimage.io

import time

from tqdm.notebook import tqdm
tqdm.pandas()

import shutil

from pydub import AudioSegment

from joblib import Parallel, delayed

import librosa

import io
import soundfile as sf

In [123]:
# extract_path = '/content/input/dlsprint'
TRAIN_PATH = os.path.join(extract_path,'train_files') 
TEST_PATH = os.path.join(extract_path,'test_files')
VALIDATION_PATH = os.path.join(extract_path,'validation_files')


In [124]:
errors=["common_voice_bn_31727562",
        'common_voice_bn_30998934',
        'common_voice_bn_31595526',
        'common_voice_bn_31534853',
        'common_voice_bn_31518061',
        'common_voice_bn_31518373',
        'common_voice_bn_31613621',
        'common_voice_bn_31555333',
        'common_voice_bn_31772113',
        'common_voice_bn_31605391',
        'common_voice_bn_31631175',
        'common_voice_bn_31563901',
        'common_voice_bn_31691690',
        'common_voice_bn_31692010',
        'common_voice_bn_31683653',
        'common_voice_bn_31692182',
        'common_voice_bn_31519976',
        'common_voice_bn_31675793',
        'common_voice_bn_31019914',
        'common_voice_bn_31660287',
        'common_voice_bn_31660384',
        'common_voice_bn_31557261',
        'common_voice_bn_31633101',
        'common_voice_bn_31599243',
        'common_voice_bn_31521515',
        'common_voice_bn_31777802',
        'common_voice_bn_31777848',
        'common_voice_bn_31669646',
        'common_voice_bn_31566083',
        'common_voice_bn_31530331',
        'common_voice_bn_31727697',
        'common_voice_bn_31513270',
        'common_voice_bn_31686295',
        'common_voice_bn_31753693',
        'common_voice_bn_31686334',
        'common_voice_bn_31765546',
        'common_voice_bn_31765548',
        'common_voice_bn_31662742',
        'common_voice_bn_31704856',
        'common_voice_bn_31635344',
        'common_voice_bn_31618327',
        'common_voice_bn_31743074',
        'common_voice_bn_31678862',
        'common_voice_bn_31626674',
        'common_voice_bn_31626677',
        'common_voice_bn_31523889',
        'common_voice_bn_31610804',
        'common_voice_bn_31769538',
        'common_voice_bn_31533273',
        'common_voice_bn_31445621',
        'common_voice_bn_31620650']
#---------------
# data filtering
#---------------
# def filter_votes(x):
#     p=x["path"]
#     # avoid error data
#     for pe in errors:
#         if pe in p:
#             return None
#     up=x["up_votes"]
#     down=x["down_votes"]
#     # if up-down<=0:
#     if up<down:
#         return None
#     # elif up==0:
#     #     return None
#     else:
#         return up

def filter_votes(x):
    p=x["path"]
    # avoid error data
    for pe in errors:
        if pe in p:
            return None
    # now process votes
    up=x["up_votes"]
    down=x["down_votes"]
    if up-down<=0:
        return "unv"
    elif up==0:
        return "unv"
    else:
        return up

In [125]:
print("Total Data before filtering:",len(train_df))
train_df["up_votes"]=train_df.progress_apply(lambda x:filter_votes(x),axis=1)
train_df.dropna(subset = ['up_votes'],inplace=True)
train_df=train_df.loc[train_df.up_votes!="unv"]
train_df.reset_index(drop=True,inplace=True)
print("Total Data after filtering:",len(train_df))
train_audio_files=train_df["path"].tolist()
train_df.to_csv('new_train.csv')

Total Data before filtering: 206950


  0%|          | 0/206950 [00:00<?, ?it/s]

Total Data after filtering: 37403


## Creating a list of test files

In [126]:
# list to store files
test_audio_files = []

# Iterate directory
for path in os.listdir(TEST_PATH):
    # check if current path is a file
    if os.path.isfile(os.path.join(TEST_PATH, path)):
        test_audio_files.append(path)
len(test_audio_files)

7747

## Creating a list of validation files

In [127]:
# list to store files
val_audio_files = []

# Iterate directory
for path in os.listdir(VALIDATION_PATH):
    # check if current path is a file
    if os.path.isfile(os.path.join(VALIDATION_PATH, path)):
        val_audio_files .append(path)
len(val_audio_files)

7747

## Declearing the folder path for converted files


In [128]:
# extract_path = '../data/dlsprint'
# drive_path = '/content/gdrive/MyDrive/kaggleProject/dlsprint'
new_dir = ['train_wavs','validation_files_wav','test_files_wav']
for folder in new_dir:
    newFolder = os.path.join(extract_path,folder)
    # newFolder = os.path.join(drive_path,folder) 
    if not os.path.exists(newFolder):
        print(newFolder)
        os.makedirs(newFolder)

../data/dlsprint/test_files_wav


In [129]:
# extract_path = '/content/input/dlsprint'
# drive_path = '/content/gdrive/MyDrive/kaggleProject/dlsprint'
New_Train_Dir = os.path.join(extract_path, new_dir[0]) # './new_train' 
NEW_Validation_Dir = os.path.join(extract_path, new_dir[1]) # './new_validation'
New_Test_Dir = os.path.join(extract_path, new_dir[2]) # './new_test'

# # For save to drive 
# New_Train_Dir = os.path.join(drive_path,'new_train') # './new_train' 
# NEW_Validation_Dir = os.path.join(drive_path,'new_validation') # './new_validation'
# New_Test_Dir = os.path.join(drive_path,'new_test') # './new_test'

In [130]:
print(New_Train_Dir)

../data/dlsprint/train_wavs


## Converting the train .mp3 files into .wav and saving it inside a new directory

In [131]:
def save_fn(filename):
    
    
    path = f"{TRAIN_PATH}/{filename}"
    save_path = f"{New_Train_Dir}"
    if not os.path.exists(save_path):
        os.makedirs(save_path, exist_ok=True)
    
    if os.path.exists(path):
        try:
            sound = AudioSegment.from_mp3(path)
            sound = sound.set_frame_rate(16000)
            sound.export(f"{save_path}/{filename[:-4]}.wav", format="wav")
        except:
            print(path)

In [132]:
import os, platform, subprocess, re

def get_processor_name():
    if platform.system() == "Windows":
        return platform.processor()
    elif platform.system() == "Darwin":
        os.environ['PATH'] = os.environ['PATH'] + os.pathsep + '/usr/sbin'
        command ="sysctl -n machdep.cpu.brand_string"
        return subprocess.check_output(command).strip()
    elif platform.system() == "Linux":
        command = "cat /proc/cpuinfo"
        all_info = subprocess.check_output(command, shell=True).decode().strip()
        for line in all_info.split("\n"):
            if "model name" in line:
                return re.sub( ".*model name.*:", "", line,1)
    return ""

get_processor_name()

' 11th Gen Intel(R) Core(TM) i7-11700 @ 2.50GHz'

In [133]:
import time
start = time.time()

Parallel(n_jobs=-1, backend="multiprocessing")(
    delayed(save_fn)(filename) for filename in tqdm(train_audio_files)
)

end = time.time()
print("total time to process: {x} seconds".format(x=end-start))

  0%|          | 0/37403 [00:00<?, ?it/s]

KeyboardInterrupt: 

## Converting the validation .mp3 files into .wav and saving it inside a new directory

In [ ]:
def val_save_fn(filename):
    
    
    path = f"{VALIDATION_PATH}/{filename}"
    save_path = f"{NEW_Validation_Dir}"
    if not os.path.exists(save_path):
        os.makedirs(save_path, exist_ok=True)
    
    if os.path.exists(path):
        try:
            sound = AudioSegment.from_mp3(path)
            sound = sound.set_frame_rate(16000)
            sound.export(f"{save_path}/{filename[:-4]}.wav", format="wav")
        except:
            print(path)

In [ ]:
start = time.time()

Parallel(n_jobs=-1, backend="multiprocessing")(
    delayed(val_save_fn)(filename) for filename in tqdm(val_audio_files)
)

end = time.time()
print("total time to process: {x} seconds".format(x=end-start))

## Converting the test .mp3 files into .wav and saving it inside a new directory

In [ ]:
def test_save_fn(filename):
    
    
    path = f"{TEST_PATH}/{filename}"
    save_path = f"{New_Test_Dir}"
    if not os.path.exists(save_path):
        os.makedirs(save_path, exist_ok=True)
    
    if os.path.exists(path):
        try:
            sound = AudioSegment.from_mp3(path)
            sound = sound.set_frame_rate(16000)
            sound.export(f"{save_path}/{filename[:-4]}.wav", format="wav")
        except:
            print(path)

In [ ]:
start = time.time()

Parallel(n_jobs=-1, backend="multiprocessing")(
    delayed(test_save_fn)(filename) for filename in tqdm(test_audio_files)
)

end = time.time()
print("total time to process: {x} seconds".format(x=end-start))